In [1]:
from flask import Flask, redirect, request, render_template, url_for, send_from_directory, make_response
from werkzeug.wrappers import Request, Response
from werkzeug.serving import run_simple
from jsonrpc import JSONRPCResponseManager, dispatcher
import os
import hashlib
import pandas as pd
import numpy as np
from werkzeug.utils import secure_filename
import pickle
import re
from sklearn.neural_network import MLPClassifier
from fastText import load_model
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from flask_cors import CORS
from nltk import ngrams
import json

In [2]:
pd.__version__

'0.20.3'

In [3]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stopWords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ruochen99/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!



Install fastText by doing:<br>
#git clone https://github.com/facebookresearch/fastText.git<br>
#cd fastText<br>
#pip install .<br>
#'wiki.en.bin' needs to be in the same directory as server.py (can be downloaded from <br>
#https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.zip)<br>


In [4]:
fasttext_model = 'wiki.en.bin'
fmodel = load_model(fasttext_model)

In [5]:
UPLOAD_FOLDER = '\datasets'
ALLOWED_EXTENSIONS_CSV = set(['csv'])
ALLOWED_EXTENSIONS_JSON = set(['json'])

In [6]:
# app = Flask(__name__)
# # CORS(app)
# app.config['UPLOAD_FOLDER'] = os.path.join(app.instance_path)

In [7]:
def lower_cols(lst):
    #convert data to lowercases
    #QUESTION: will I miss any important information? 
    return [word.lower() for word in lst if isinstance(word,str)]

In [8]:
def remove_chars(lst):
    #remove punctuation characters such as ",", "(", ")", """, ":", "/", and "."
    #NOTE: PRESERVES WHITE SPACE.
    #QUESTION: any other characters we should be aware of? Is this a good idea? I'm inspecting each word individually.
    #Any potential pitfalls? 
    cleaned = [re.sub('\s+', ' ', mystring).strip() for mystring in lst]
    cleaned = [re.sub(r'[[^A-Za-z0-9\s]+]', ' ', mystr) for mystr in cleaned]
    cleaned = [mystr.replace('_', ' ') for mystr in cleaned]
    return cleaned

In [9]:
def clean_cols(data):
    data = lower_cols(data)
    data = remove_chars(data)
    return data

In [10]:
def fill_empty_cols(df):
    empty_cols = []
    for i in df.columns.values:
        if (len(df[i].dropna()) == 0):
            df.at[2,i] = 1
            empty_cols.append(df.columns.get_loc(i))
    return df, empty_cols

In [39]:
def preprocess(pandas_dataset, df_target):
    if (not pandas_dataset.empty):
        organization = 'HDX'   #Replace if datasets contains organization
        pandas_dataset.dropna(how = 'all', inplace = True)
        pandas_dataset, empty_cols = fill_empty_cols(pandas_dataset)
        print(empty_cols)
#         pandas_dataset.dropna(axis=1, how = 'all', subset=range(1,len(pandas_dataset)), inplace = True)
        headers = list(pandas_dataset.columns.values)        
        headers = clean_cols(headers)
    for i in range(len(headers)):
        try:
            dic = {'Header': headers[i], 
                   'Data': list(pandas_dataset.iloc[1:, i]), 
                   'Relative Column Position': (i+1) / len(pandas_dataset.columns), 
                   'Organization': organization,
                   'Index': i}
            df_target.loc[len(df_target)] = dic
        except:
            raise Exception("Error: arguments not matched")
    df_result = transform_vectorizers(df_target)
    return df_target, df_result, empty_cols

In [12]:
def transform_vectorizers(df_target):
    number_of_data_point_to_vectorize = 7
    cols = ['Header_embedding', 'Organization_embedded', 'features_combined']
    df = pd.DataFrame(columns = cols)
    df_target, number_of_data_point_to_vectorize = embedded_datapoints(df_target, 7)
    df['data_combined'] = df_target.loc[:, 'embedded_datapoint0': 'embedded_datapoint' 
                                                           + str(number_of_data_point_to_vectorize-1)].values.tolist()
    df['data_combined'] = df['data_combined'].apply(lambda x: [val for item in x for val in item])
    df['Header_embedding'] = df_target['Header'].astype(str).apply(fmodel.get_sentence_vector)
    df['Organization_embedded'] = df_target['Organization'].astype(str).apply(fmodel.get_sentence_vector)
    cols = ['Header_embedding', 'Organization_embedded', 'data_combined']
    df['features_combined'] = df[cols].values.tolist()
    df['features_combined'] = df['features_combined'].apply(lambda x: [val for item in x for val in item])
    diff = 2700 - len(df['features_combined'][0])
    for i in range(len(df)):
        for j in range(diff):
            df['features_combined'][i].append(0)
    df = df.dropna()
    return df

In [13]:
def separate_words(series): 
    #each series is a long string that contains all the data
    lst = []
    cleanlist = [str(x) for x in series if str(x) != 'nan']
    for i in cleanlist:
        lst = re.split(r"\W+", i)
        lst.extend(list(filter(None, lst)))
    return lst
    
def vectorize_n_datapoints(df, number_of_datapoints_to_vectorize = 7):
#     print(df['Data'].head())
#     print(df['Data'].iloc[0])
#     for i in range(len(df['Data'])):
#         df['Data_separated'].iloc[0] = separate_words(df['Data'].iloc[0])
    df['Data_separated'] = df['Data'].apply(separate_words)
    if (number_of_datapoints_to_vectorize > len(df['Data_separated'][0])):
        number_of_datapoints_to_vectorize = len(df['Data_separated'][0])
    for i in range(number_of_datapoints_to_vectorize):
        df['datapoint' + str(i)] = df['Data_separated'].str[i]
    return df, number_of_datapoints_to_vectorize

In [14]:
def embedded_datapoints(df, number_of_data_point_to_vectorize=7):
    df, number_of_data_point_to_vectorize = vectorize_n_datapoints(df)
    for i in range(number_of_data_point_to_vectorize):
        
        df['embedded_datapoint' + str(i)] = df['datapoint' + str(i)].map(lambda x: fmodel.get_sentence_vector(str(x)))
    return df, number_of_data_point_to_vectorize

In [15]:
def remove_stop_words(data_lst):
    #remove stopwords from the data including 'the', 'and' etc.
    wordsFiltered = []
    for w in data_lst:
        if w not in stopWords:
            wordsFiltered.append(w)
    return wordsFiltered

In [16]:
def word_extract(row):
    ignore = ['nan']
    no_white = [i.lstrip() for i in row if i not in ignore and not (isinstance(i, float) or isinstance(i,int))]
    cleaned_text = [w.lower() for w in no_white if w not in ignore]
    return cleaned_text

In [17]:
def allowed_file_csv(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS_CSV

In [18]:
def allowed_file_json(filename):
    return '.' in filename and \
            filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS_JSON

In [19]:
def generate_n_grams(data_lst, n):
    # cleaned = remove_chars(list(data_lst))
    # cleaned = clean_cols(cleaned)
    cleaned = remove_stop_words(data_lst)
    #make sure that n_grams 'refresh' when a new dataset is encountered!!!!   
    return list(ngrams(cleaned, n))

In [20]:
input_dataset = pd.read_csv('test_files/fts_incoming_funding_tgo.csv', encoding = "ISO-8859-1", na_values=['nan',' nan'])
input_headers = input_dataset.columns.values
# input_dataset = input_dataset.rename(columns=input_dataset.iloc[0]).drop(input_dataset.index[0])

In [21]:
input_dataset.head()

,date,budgetYear,description,amountUSD,srcOrganization,srcOrganizationTypes,srcLocations,srcUsageYearStart,srcUsageYearEnd,destPlan,...,firstReportedDate,decisionDate,keywords,originalAmount,originalCurrency,exchangeRate,id,refCode,createdAt,updatedAt
0,#date,#date+year+budget,#description+notes,#value+funding+total+usd,#org+name+funder,#org+type+funder+list,#country+iso3+funder+list,#date+year+start+funder,#date+year+end+funder,#activity+appeal+name+impl,...,#date+reported,#date+decision,#description+keywords,#value+funding+total,#value+funding+total+currency,#financial+fx,#activity+id+fts_internal,#activity+code,#date+created,#date+updated
1,2019-03-08,NaN,Fall Army Worm / Lutte contre la chenille legi...,1000000,African Development Bank,Inter-governmental,NaN,2019,2019,NaN,...,2019-03-14,2019-03-08,NaN,NaN,USD,NaN,188251,OSRO/TOG/901/ADB,2019-03-15,2019-03-18
2,2017-01-31,NaN,Towards a Universal Access of Vulnerable Girls...,0,Education Above All Foundation,Private organization/foundation,NaN,2017,2017,NaN,...,2018-05-20,2017-01-31,Multiyear,NaN,NaN,NaN,175161,EAA/EAC/CT/16/2016,2018-05-23,2018-05-23
3,2017-01-31,NaN,Towards a Universal Access of Vulnerable Girls...,183965,Education Above All Foundation,Private organization/foundation,NaN,2017,2017,NaN,...,2018-05-20,2017-01-31,Multiyear,NaN,NaN,NaN,175165,EAA/EAC/CT/16/2016,2018-05-23,2018-05-23


In [40]:
# process the untagged dataset
raw, processed_dataset, empty_cols = preprocess(input_dataset, 
                               pd.DataFrame(columns=['Header','Data','Relative Column Position','Organization','Index']))

[]


In [41]:
processed_dataset.head()

,Header_embedding,Organization_embedded,features_combined,data_combined
0,"[0.00187157, -0.0188091, -0.00182546, -0.03487...","[0.0122939, -0.0148763, -0.0858311, 0.0693863,...","[0.00187157, -0.0188091, -0.00182546, -0.03487...","[0.061183, -0.108557, -0.0215283, -0.0257585, ..."
1,"[-0.0577098, 0.0106014, 0.076496, 0.0175093, 0...","[0.0122939, -0.0148763, -0.0858311, 0.0693863,...","[-0.0577098, 0.0106014, 0.076496, 0.0175093, 0...","[0.0566073, 0.0195136, -0.00766985, -0.0078007..."
2,"[-0.0117924, 0.0334399, -0.0723287, 0.00917512...","[0.0122939, -0.0148763, -0.0858311, 0.0693863,...","[-0.0117924, 0.0334399, -0.0723287, 0.00917512...","[-0.0609063, -0.0311797, -0.031652, 0.08444, 0..."
3,"[-0.0607706, -0.0288627, -0.0202762, -0.003495...","[0.0122939, -0.0148763, -0.0858311, 0.0693863,...","[-0.0607706, -0.0288627, -0.0202762, -0.003495...","[0.028683, 0.0406691, -0.0608491, -0.00199248,..."
4,"[-0.0170964, -0.0967856, -0.00234281, 0.035446...","[0.0122939, -0.0148763, -0.0858311, 0.0693863,...","[-0.0170964, -0.0967856, -0.00234281, 0.035446...","[0.0490048, -0.0140489, -0.0217238, 0.0548013,..."


In [ ]:
na = []
for i in range(len(processed_dataset)):
    na.append(' ')

In [63]:
def tag_predicted(clf, X_test, series, threshold):
    #True if tag should be left blank
    if (not isinstance(X_test, np.ndarray)):
        X_test = X_test.values.tolist()
    probs = clf.predict_proba(X_test)
    values = []
    for i in range(len(X_test)):
        max_arg = probs[i].argsort()[-1]
        top_suggested_tag = clf.classes_[max_arg]
        prob = np.take(probs[i], max_arg)
        if (prob > threshold):
            values.append(False)
        else:
            values.append(True)
    return values

#helper function to fill in the blanks for tags that have a confidence level less than the threshold
def fill_blank_tags(predicted_tags, clf, X_test, series, threshold = 0.5):
    boolean_array = tag_predicted(clf, X_test, series, threshold)
    for i in range(len(predicted_tags)):
        if (boolean_array[i] == True):
            predicted_tags[i] = ''
    return predicted_tags

In [120]:
model = pickle.load(open("model.pkl", "rb")) #Model needs be named model.pkl
output_dataset = pd.DataFrame(data = model.predict(list(processed_dataset['features_combined'])))


/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator LabelBinarizer from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator MLPClassifier from version 0.19.0 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [121]:
output_dataset = fill_blank_tags(output_dataset.iloc[:, 0].values, model, processed_dataset["features_combined"], raw['Header'])

In [122]:
def add_hashtags(predicted_tags):
    result = []
    if (isinstance(predicted_tags, np.ndarray)):
        for word in predicted_tags:
            if word == '':
                result.append('')
            else:
                result.append("#"+word)
    return result

In [123]:
output_dataset = add_hashtags(output_dataset)
output_dataset = pd.DataFrame(add_hashtags(output_dataset))

In [134]:
output_dataset.iloc[:, 0].values

array(['#date', '#date', '', '#affected', '#org', '#org', '', '#date',
       '#date', '', '', '#affected', '#org', '', '', '#country', '', '',
       '#date', '#date', '#affected', '', '#affected', '', '#status',
       '#date', '#date', '', '#affected', '#affected', '#affected',
       '#meta', '', '#date', '#date'], dtype=object)

In [138]:
input_dataset.loc[-1] = output_dataset.iloc[:, 0].values
input_dataset.index = input_dataset.index + 1
input_dataset = input_dataset.sort_index()
input_dataset

,date,budgetYear,description,amountUSD,srcOrganization,srcOrganizationTypes,srcLocations,srcUsageYearStart,srcUsageYearEnd,destPlan,...,firstReportedDate,decisionDate,keywords,originalAmount,originalCurrency,exchangeRate,id,refCode,createdAt,updatedAt
0,#date,#date,,#affected,#org,#org,,#date,#date,,...,#date,#date,,#affected,#affected,#affected,#meta,,#date,#date
1,#date,#date,,#affected,#org,#org,,#date,#date,,...,#date,#date,,#affected,#affected,#affected,#meta,,#date,#date
2,#date,#date+year+budget,#description+notes,#value+funding+total+usd,#org+name+funder,#org+type+funder+list,#country+iso3+funder+list,#date+year+start+funder,#date+year+end+funder,#activity+appeal+name+impl,...,#date+reported,#date+decision,#description+keywords,#value+funding+total,#value+funding+total+currency,#financial+fx,#activity+id+fts_internal,#activity+code,#date+created,#date+updated
3,2019-03-08,NaN,Fall Army Worm / Lutte contre la chenille legi...,1000000,African Development Bank,Inter-governmental,NaN,2019,2019,NaN,...,2019-03-14,2019-03-08,NaN,NaN,USD,NaN,188251,OSRO/TOG/901/ADB,2019-03-15,2019-03-18
4,2017-01-31,NaN,Towards a Universal Access of Vulnerable Girls...,0,Education Above All Foundation,Private organization/foundation,NaN,2017,2017,NaN,...,2018-05-20,2017-01-31,Multiyear,NaN,NaN,NaN,175161,EAA/EAC/CT/16/2016,2018-05-23,2018-05-23
5,2017-01-31,NaN,Towards a Universal Access of Vulnerable Girls...,183965,Education Above All Foundation,Private organization/foundation,NaN,2017,2017,NaN,...,2018-05-20,2017-01-31,Multiyear,NaN,NaN,NaN,175165,EAA/EAC/CT/16/2016,2018-05-23,2018-05-23


In [109]:
output_dataset.loc[empty_cols,0] = 'No Prediction. Column only had missing values'
output_dataset.insert(loc=0, column='Header', value=input_headers)
# output_dataset.insert(loc=1, column='Original tag', value=np.array(input_dataset.iloc[0,:]))
output_dataset.insert(loc=1, column='Original tag', value=na)
output_dataset.rename(index=str, columns={0: "Predicted tag"}, inplace=True)
output_dataset

NameError: name 'na' is not defined

In [ ]:
import pandas
from openpyxl import load_workbook

book = load_workbook('compare.xlsx')
writer = pandas.ExcelWriter('compare.xlsx', engine='openpyxl')
writer.book = book
writer.sheets = {ws.title: ws for ws in book.worksheets}

for sheetname in writer.sheets:
    output_dataset.to_excel(writer,sheet_name=sheetname, startrow=writer.sheets[sheetname].max_row, index = False,header= False)

writer.save()

In [ ]:
# @app.route('/', methods=['GET','POST'])
# def upload_file():
#     if request.method == 'POST':
#         # check if the post request has the file part
#         if 'file' not in request.files:
#             flash('No file part')
#             return redirect(request.url)
#         file = request.files['file']
        
#         if file.filename == '':
#             # flash('No selected file')
#             return redirect(request.url)
#         # file.save(os.getcwd())
#         if file and allowed_file_csv(file.filename):
#             filename = secure_filename(file.filename)
#             input_dataset = pd.read_csv(file)
                
#         if file and allowed_file_json(file.filename):
#             # filename = secure_filename(file.filename)
#             input_dataset = pd.read_json(file)
#             input_dataset = input_dataset.rename(columns=input_dataset.iloc[0]).drop(input_dataset.index[0])
#                 # process the untagged dataset
#         processed_dataset = preprocess(input_dataset, 
#             pd.DataFrame(columns=['Header','Data','Relative Column Position','Organization','Index']))
#         model = pickle.load(open("model.pkl", "rb")) #Model needs be named model.pkl, preferably using version 0.20.3
#         output_dataset = pd.DataFrame(data = model.predict(list(processed_dataset['features_combined'])))
#         resp = make_response(output_dataset.to_csv())
#         resp.headers["Content-Disposition"] = "attachment; filename=export.csv"
#         resp.headers["Content-Type"] = "text/csv"
#         return resp
        
          

    return 
<br>
    <!doctype html><br>
    <title>Upload new File</title><br>
    <h1>Upload new File (only CSV and JSON files accepted)</h1><br>
    <form method=post enctype=multipart/form-data><br>
      <input type=file name=file><br>
      <input type=submit value=Upload><br>
    # <form method=post><br>
    #   <input name=text><br>
    #   <input type=submit><br>
    </form><br>
  
 

In [ ]:
model = pickle.load(open("model.pkl", "rb")) #Model needs be named model.pkl
output_dataset = pd.DataFrame(data = model.predict(list(processed_dataset['features_combined'])))
output_dataset.loc[empty_cols,0] = 'No Prediction. Column only had missing values'
output_dataset.insert(loc=0, column='Header', value=input_headers)
output_dataset.insert(loc=1, column='original tag', value=input_dataset.iloc[0,:].values)
output_dataset.rename(index=str, columns={0: "Predicted tag"}, inplace=True)
output_dataset

In [ ]:
# if __name__ == '__main__':
#      app.run(debug=True)
     

In [ ]:
processed_dataset

In [ ]:
input_dataset